In [ ]:
!pip install --upgrade transformers

In [8]:
from pypdf import PdfReader
from docx import Document

In [6]:
!pip install pypdf
!pip install python-docx

In [1]:
from huggingface_hub import login
login("enter your hugging face api or access key")

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def extract_text_from_docx(docx_path):
    text = ""
    doc = Document(docx_path)
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

file_path ="path to your resume " # change pfd or docx accordingly


if file_path.endswith('.pdf'):
    extracted_text = extract_text_from_pdf(file_path)
elif file_path.endswith('.docx'):
    extracted_text = extract_text_from_docx(file_path)
else:
    raise ValueError("Unsupported file type, Only pdf or docx is acceptrd ")

print(extracted_text)
messages = [
    {"role": "system", "content": "You are an AI bot designed to act as a professional for parsing resumes. You are given with resume and your job is to extract the following information from the resume 1. full name 2. email id 3. phone number 4. technical skills 5. employment details 6. college details, school details 7. certifications 8. projects (only name of projects, dont show responsibility)"},
    {"role": "user", "content": f"RESUME:{extracted_text}"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])